In [135]:
import numpy as np
import pandas as pd
from pathlib import Path
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.linear_model import LogisticRegression
# Regression Models
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import BaggingRegressor

# Classifer Models
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import BaggingClassifier

# To visualize individual decision trees
from sklearn import tree
from sklearn.tree import export_text


In [118]:
train_df = pd.read_csv(Path('Resources/2019loans.csv'))
test_df = pd.read_csv(Path('Resources/2020Q1loans.csv'))

In [119]:
train_df.head()

Unnamed: 0   index  loan_amnt  int_rate  installment home_ownership  \
0       57107   57107    13375.0    0.1797       483.34       MORTGAGE   
1      141451  141451    21000.0    0.1308       478.68       MORTGAGE   
2      321143  321143    20000.0    0.1240       448.95       MORTGAGE   
3       11778   11778     3000.0    0.1240       100.22           RENT   
4      169382  169382    30000.0    0.1612      1056.49       MORTGAGE   

   annual_inc verification_status loan_status pymnt_plan  ...  pct_tl_nvr_dlq  \
0    223000.0        Not Verified    low_risk          n  ...           100.0   
1    123000.0     Source Verified    low_risk          n  ...            85.0   
2    197000.0     Source Verified    low_risk          n  ...            85.7   
3     45000.0        Not Verified    low_risk          n  ...           100.0   
4    133000.0     Source Verified    low_risk          n  ...           100.0   

   percent_bc_gt_75  pub_rec_bankruptcies  tax_liens  tot_hi_cred_lim  \
0              50.0                   0.0        0.0         577150.0   
1              33.3                   0.0        0.0         132750.0   
2              33.3                   0.0        0.0         628160.0   
3              16.7                   1.0        0.0          42006.0   
4              66.7                   0.0        0.0         283248.0   

   total_bal_ex_mort  total_bc_limit total_il_high_credit_limit  \
0           122018.0         32000.0                   170200.0   
1            27896.0         15900.0                    35398.0   
2           114043.0         22600.0                    90340.0   
3            20761.0         19900.0                    15406.0   
4           109056.0         79500.0                    58778.0   

   hardship_flag  debt_settlement_flag  
0              N                     N  
1              N                     N  
2              N                     N  
3              N                     N  
4              N                     N  

[5 rows x 86 columns]

In [120]:
test_df.head()

Unnamed: 0  index  loan_amnt  int_rate  installment home_ownership  \
0       67991  67991    40000.0    0.0819       814.70       MORTGAGE   
1       25429  25429     6000.0    0.1524       208.70           RENT   
2       38496  38496     3600.0    0.1695       128.27           RENT   
3       19667  19667    20000.0    0.1524       478.33           RENT   
4       37505  37505     3600.0    0.1240       120.27           RENT   

   annual_inc verification_status loan_status pymnt_plan  ...  pct_tl_nvr_dlq  \
0    140000.0        Not Verified    low_risk          n  ...            97.7   
1     55000.0        Not Verified    low_risk          n  ...            66.7   
2     42000.0        Not Verified    low_risk          n  ...           100.0   
3    100000.0        Not Verified    low_risk          n  ...           100.0   
4     50000.0        Not Verified    low_risk          n  ...           100.0   

   percent_bc_gt_75  pub_rec_bankruptcies  tax_liens  tot_hi_cred_lim  \
0               0.0                   0.0        0.0         527975.0   
1               0.0                   0.0        0.0          34628.0   
2               0.0                   0.0        0.0          23100.0   
3              50.0                   0.0        0.0          56481.0   
4              25.0                   0.0        0.0          45977.0   

   total_bal_ex_mort  total_bc_limit total_il_high_credit_limit  \
0            70914.0         74600.0                    99475.0   
1            23460.0          5900.0                    23628.0   
2            19183.0          7300.0                    15000.0   
3            43817.0         13800.0                    35981.0   
4            32448.0         21000.0                    24977.0   

   hardship_flag  debt_settlement_flag  
0              N                     N  
1              N                     N  
2              N                     N  
3              N                     N  
4              N                     N  

[5 rows x 86 columns]

In [121]:
# Convert categorical data to numeric and separate target feature for training data
train_features = train_df.drop('loan_status', axis=1)

In [122]:
# One-hot encoding the entire dataframe
X_train = pd.get_dummies(train_features)
X_train.head()


Unnamed: 0   index  loan_amnt  int_rate  installment  annual_inc    dti  \
0       57107   57107    13375.0    0.1797       483.34    223000.0  29.99   
1      141451  141451    21000.0    0.1308       478.68    123000.0  11.26   
2      321143  321143    20000.0    0.1240       448.95    197000.0  11.28   
3       11778   11778     3000.0    0.1240       100.22     45000.0  18.08   
4      169382  169382    30000.0    0.1612      1056.49    133000.0  27.77   

   delinq_2yrs  inq_last_6mths  open_acc  ...  verification_status_Verified  \
0          0.0             0.0      15.0  ...                             0   
1          2.0             0.0      16.0  ...                             0   
2          0.0             0.0      12.0  ...                             0   
3          0.0             0.0      12.0  ...                             0   
4          0.0             2.0      13.0  ...                             0   

   pymnt_plan_n  initial_list_status_f  initial_list_status_w  \
0             1                      0                      1   
1             1                      0                      1   
2             1                      0                      1   
3             1                      0                      1   
4             1                      0                      1   

   application_type_Individual  application_type_Joint App  hardship_flag_N  \
0                            1                           0                1   
1                            1                           0                1   
2                            1                           0                1   
3                            1                           0                1   
4                            1                           0                1   

   hardship_flag_Y  debt_settlement_flag_N  debt_settlement_flag_Y  
0                0                       1                       0  
1                0                       1                       0  
2                0                       1                       0  
3                0                       1                       0  
4                0                       1                       0  

[5 rows x 94 columns]

In [123]:
target_train=train_df['loan_status']
y_train = LabelEncoder().fit_transform(train_df['loan_status'])
y_train

array([1, 1, 1, ..., 0, 0, 0])

In [124]:
# Convert categorical data to numeric and separate target feature for testing data
test_features = test_df.drop('loan_status', axis=1)
test_features.head()

Unnamed: 0  index  loan_amnt  int_rate  installment home_ownership  \
0       67991  67991    40000.0    0.0819       814.70       MORTGAGE   
1       25429  25429     6000.0    0.1524       208.70           RENT   
2       38496  38496     3600.0    0.1695       128.27           RENT   
3       19667  19667    20000.0    0.1524       478.33           RENT   
4       37505  37505     3600.0    0.1240       120.27           RENT   

   annual_inc verification_status pymnt_plan    dti  ...  pct_tl_nvr_dlq  \
0    140000.0        Not Verified          n  19.75  ...            97.7   
1     55000.0        Not Verified          n  11.52  ...            66.7   
2     42000.0        Not Verified          n   6.74  ...           100.0   
3    100000.0        Not Verified          n  12.13  ...           100.0   
4     50000.0        Not Verified          n  16.08  ...           100.0   

   percent_bc_gt_75  pub_rec_bankruptcies  tax_liens  tot_hi_cred_lim  \
0               0.0                   0.0        0.0         527975.0   
1               0.0                   0.0        0.0          34628.0   
2               0.0                   0.0        0.0          23100.0   
3              50.0                   0.0        0.0          56481.0   
4              25.0                   0.0        0.0          45977.0   

   total_bal_ex_mort total_bc_limit  total_il_high_credit_limit  \
0            70914.0        74600.0                     99475.0   
1            23460.0         5900.0                     23628.0   
2            19183.0         7300.0                     15000.0   
3            43817.0        13800.0                     35981.0   
4            32448.0        21000.0                     24977.0   

   hardship_flag  debt_settlement_flag  
0              N                     N  
1              N                     N  
2              N                     N  
3              N                     N  
4              N                     N  

[5 rows x 85 columns]

In [125]:
# add missing dummy variables to testing set
# One-hot encoding the entire test dataframe
X_test= pd.get_dummies(test_features)
X_test.head()


Unnamed: 0  index  loan_amnt  int_rate  installment  annual_inc    dti  \
0       67991  67991    40000.0    0.0819       814.70    140000.0  19.75   
1       25429  25429     6000.0    0.1524       208.70     55000.0  11.52   
2       38496  38496     3600.0    0.1695       128.27     42000.0   6.74   
3       19667  19667    20000.0    0.1524       478.33    100000.0  12.13   
4       37505  37505     3600.0    0.1240       120.27     50000.0  16.08   

   delinq_2yrs  inq_last_6mths  open_acc  ...  \
0          0.0             1.0      18.0  ...   
1          2.0             0.0       8.0  ...   
2          0.0             0.0       6.0  ...   
3          0.0             2.0       7.0  ...   
4          0.0             3.0       6.0  ...   

   verification_status_Source Verified  verification_status_Verified  \
0                                    0                             0   
1                                    0                             0   
2                                    0                             0   
3                                    0                             0   
4                                    0                             0   

   pymnt_plan_n  initial_list_status_f  initial_list_status_w  \
0             1                      0                      1   
1             1                      0                      1   
2             1                      0                      1   
3             1                      0                      1   
4             1                      0                      1   

   application_type_Individual  application_type_Joint App  hardship_flag_N  \
0                            1                           0                1   
1                            1                           0                1   
2                            1                           0                1   
3                            1                           0                1   
4                            1                           0                1   

   hardship_flag_Y  debt_settlement_flag_N  
0                0                       1  
1                0                       1  
2                0                       1  
3                0                       1  
4                0                       1  

[5 rows x 93 columns]

In [126]:
# y_test
target_test=test_df['loan_status']
y_test = LabelEncoder().fit_transform(test_df['loan_status'])
print(y_test)

[1 1 1 ... 0 0 0]


In [127]:
for column in X_train.columns:
    if column not in X_test.columns:
        X_test[column] = 0

In [128]:
# Train the Logistic Regression model on the unscaled data and print the model score
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression(solver='lbfgs', max_iter=30000)
clf.fit(X_train, y_train)

LogisticRegression(max_iter=30000)

In [129]:
print('Training accuracy:', clf.score(X_train, y_train))
print('Test accuracy:', clf.score(X_test, y_test))

Training accuracy: 0.7027914614121511
Test accuracy: 0.5642279880901744


In [140]:
# Train a Random Forest Classifier model and print the model score
# Classifer Models
from sklearn.ensemble import RandomForestClassifier

clf_n = RandomForestClassifier(n_estimators=100)
clf_n.fit(X_train, y_train)



RandomForestClassifier()

In [142]:
print('Random Forest Training accuracy:', clf_n.score(X_train, y_train))
print('Random Forest Test accuracy:', clf_n.score(X_test, y_test))

Random Forest Training accuracy: 1.0
Random Forest Test accuracy: 0.6284559761803488


In [131]:
# Scale the data

# Scaling the X data by using StandardScaler()
scaler = StandardScaler().fit(X_train)
X_train_scaled = scaler.transform(X_train)
X_train_scaled

array([[-1.31172014, -1.31172014, -0.39311205, ..., -0.17149859,
         0.02026518, -0.02026518],
       [-0.46579523, -0.46579523,  0.35168119, ..., -0.17149859,
         0.02026518, -0.02026518],
       [ 1.3364188 ,  1.3364188 ,  0.25400339, ..., -0.17149859,
         0.02026518, -0.02026518],
       ...,
       [ 1.67571549,  1.67571549, -1.34791257, ..., -0.17149859,
         0.02026518, -0.02026518],
       [ 1.67600634,  1.67600634, -0.23438563, ..., -0.17149859,
         0.02026518, -0.02026518],
       [ 1.67906533,  1.67906533, -0.23438563, ..., -0.17149859,
         0.02026518, -0.02026518]])

In [132]:
# Transforming the test dataset based on the fit from the training dataset
X_test_scaled = scaler.transform(X_test)
X_test_scaled

array([[-1.20255948, -1.20255948,  2.20755943, ..., -0.17149859,
         0.02026518, -0.02026518],
       [-1.62943343, -1.62943343, -1.11348584, ..., -0.17149859,
         0.02026518, -0.02026518],
       [-1.49837845, -1.49837845, -1.34791257, ..., -0.17149859,
         0.02026518, -0.02026518],
       ...,
       [-1.10927546, -1.10927546, -0.72277464, ..., -0.17149859,
         0.02026518, -0.02026518],
       [-1.10922531, -1.10922531, -0.91813024, ..., -0.17149859,
         0.02026518, -0.02026518],
       [-1.1091551 , -1.1091551 ,  1.23078141, ..., -0.17149859,
         0.02026518, -0.02026518]])

In [133]:
# Train the Logistic Regression model on the scaled data and print the model score

In [134]:
# Train a Random Forest Classifier model on the scaled data and print the model score